## Assignment 3 Question 2

Is it possible to predict ELO of a player based on context of a potential en passant move?

Only investigating the player who is in the position of making the en passant move.

Lila See FDS PCA solutions for 3 good points for what PCA is good for - include in report?

#### Import Libraries

In [27]:
import pandas as pd
import chess
import chess.pgn
import io
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import time
import copy

# PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

#### Import CSV file

66879 entries in dataframe

In [4]:
# Load CSV into a big dataframe
CHESS_DATA_LOCATION = "data/club_games_data.csv"
chess_data = pd.read_csv(CHESS_DATA_LOCATION)

#### Dataframe info

Using:
print(chess_data.dtypes)
print(chess["rules"].unique())


Chess rules:
['chess' 'chess960' 'threecheck' 'crazyhouse' 'kingofthehill']

Time control:
['1/259200' '1/172800' '1800' '1/86400' '1/432000' '1/604800' '600'
 '120+1' '900+10' '300' '180+2' '3600+5' '2700+45' '3600' '1/1209600'
 '180' '600+10' '60' '480+3' '300+5' '420+3' '600+5' '600+2' '1200' '30'
 '60+1' '120' '1500+3' '900+2' '1500+5' '1500+10' '1/864000' '900' '300+2'
 '1500' '7200' '300+1' '5400' '3600+60' '2700+30' '3480+45' '10' '2700+10'
 '15' '2700' '3600+20' '4500' '4200' '900+5' '1800+10' '2700+5' '480+5'
 '1800+30' '300+3' '600+1' '1800+5' '420+5' '5400+30' '240+10' '420' '303'
 '60+10']

 Time class:
['daily' 'rapid' 'bullet' 'blitz']

#### Clean data

Undeveloped board shouldn't matter if we're filtering games for potential ep

Same for draws

Can filter our time class if are looking at time controls

After making a new move_list column, should we drop the pgn column?


Variables we are considering when predicting ELO (for a player who could potentially make en passant move) are: (Y = DONE, N = NOT DONE)
- Y: Colour who had ep opportunity (boolean)
- Y: Did they take the en passant? (boolean)
- N: Does their choice on taking/not taking support them if gaining an advantage? (numerical value for how much of an advantage it gives)
- N: Time taken to decide to capture/not capture en passant (... whatever can be time, a number in seconds ig)
- Y: Is the game rated? (boolean)
- Y: Game time class


To do:
- Work out which colour is making the potential en passant move, add a column to dataframe detailing this
- Make dataframe columns for other variables
- Apply PCA reduction

In [5]:
# Drops rows if any value is a NaN (data is clean so it doesn't do anything)
chess_data.dropna(axis=0, how='any')

# Filter out alternative rules like chess960 etc
# Important this comes before the drop irrelevant columns line
chess_data = chess_data[chess_data['rules'] == "chess"]

# Save PGN column from dataframe
full_pgn = chess_data['pgn']

def get_moves(entry):
    '''
    Retrive series of moves in a game when given the whole full_pgn entry
    '''
    pgn = entry.splitlines()[-1]
    return pgn

# Add list of moves (string) as a new column to dataframe
chess_data['move_list'] = full_pgn.apply(get_moves)

# Drop irrelevant columns
chess_data = chess_data.drop(['time_control', 'white_username', 'black_username','white_id', 'black_id', 'white_result', 'black_result', 'rules'], axis=1)




#### Import chess info
https://python-chess.readthedocs.io/en/latest/core.html#chess.Board.san


#### En Passant functions
- has_legal_en_passant() tests if en passant capturing would actually be possible on the next move.
- has_pseudo_legal_en_passant()
- has_legal_en_passant()
- is_en_passant(move: Move) Checks if the given pseudo-legal move is an en passant capture.




Use StringIO to parse games from a string.

```python
import io
pgn = io.StringIO("1. e4 e5 2. Nf3 *")
game = chess.pgn.read_game(pgn)
```

#### Clean data specifically for ep

Filter out columns that don't have potential ep
Add columns for: whether ep happened, which colour had potential to take ep

In [160]:
chess_filter2 = chess_data.copy(deep=True)

In [161]:
def check_pgn_opportunity(pgn_in):
    '''
    Checks PGN for whether opportunity for EP happened in the game.
    '''
    # print(pgn_in.splitlines()[-1])
    pgn = io.StringIO(pgn_in)       # PGN as a file
    game = chess.pgn.read_game(pgn) # Read PGN and put into game
    board = game.board()            # "board" of a game
    
    precheck = False                # Is en passant possible

    # Find only pawn moves in game
    for move in game.mainline_moves():
        # B bishop Q queen K king N knight R rook O castle ELSE pawn
        san = board.san(move)
        move_piece = san[0]
        
        # lestring = "abcdefg"
        # if [e in lestring for e in [move_piece] if e in lestring]:
        # if move_piece.startswith(("a", "b", "c", "d", "e", "f", "g")):
        # if (re.search("^[abcdefg]", move_piece) != None):
        if move_piece in "abcdefg":
            # Push the move before checking the board
            board.push(move)
            precheck = board.has_legal_en_passant()

            # Return True the moment a potential ep move has been found
            if precheck:
                return True
        else:
            board.push(move)
            continue
                
    # If ran through every move without ep opportunity then return False
    return False

def check_pgn_happened(pgn_in):
    '''
    Checks PGN for whether EP actually happened in the game.

    Assumes game passed in already has the opportunity for an EP move.
    Returns True if EP was actually the next move.
    '''
    pgn = io.StringIO(pgn_in)       # PGN as a file
    game = chess.pgn.read_game(pgn) # Read PGN and put into game
    board = game.board()            # "board" of a game
    
    moved = False                   # Was en passant moved?

    # Find only pawn moves in game
    for move in game.mainline_moves():
        # B bishop Q queen K king N knight R rook O castle ELSE pawn
        san = board.san(move)
        move_piece = san[0:1]

        if (re.search("[abcdefg]x", move_piece) != None):
            moved = board.is_en_passant(move)
            # If ep actually happened, immediately return True
            if moved:
                return True
            board.push(move)
        else:
            board.push(move)
            continue
                
    # If ran through every move without ep then return False
    return False

def check_pgn_turn(pgn_in):
    '''
    Checks which player has the opportunity for EP.

    Assumes check_pgn_opportunity is true for this pgn - otherwise None is returned.
    Returns 'White' or 'Black' depending on which player has opportunity for ep
    '''
    pgn = io.StringIO(pgn_in)       # PGN as a file
    game = chess.pgn.read_game(pgn) # Read PGN and put into game
    board = game.board()            # "board" of a game
    
    precheck = False                # Is en passant possible

    # Find only pawn moves in game
    for move in game.mainline_moves():
        # B bishop Q queen K king N knight R rook O castle ELSE pawn
        turn = board.turn
        san = board.san(move)
        move_piece = san[0]

        if move_piece in "abcdefg":
            # Push the move before checking the board
            board.push(move)
            precheck = board.has_legal_en_passant()

            # Return True the moment a potential ep move has been found
            if precheck:
                if turn:
                    return 'Black'
                else:
                    return 'White'
        else:
            board.push(move)
            continue
    # If pgn without ep opportunity was passed in, return None
    return None


def check_pgn(df_row):
    ep_set = set(["ax","bx","cx","dx","ex","fx","gx", "hx"])
    # check_set = set(["a","b","c","d","e","f","g", "h"])
    check_set = set("abcdefgh")
    # ep_set = set("axbxcxdxexfxgxhx")
    check_lookup = {"a":1,"b":1,"c":1,"d":1,"e":1,"f":1,"g":1,"h":1}
    ep_lookup = {"ax":1,"bx":1,"cx":1,"dx":1,"ex":1,"fx":1,"gx":1,"hx":1}
    '''
    Checks PGN for whether opportunity for EP happened in the game.
    '''
    # print(pgn_in.splitlines()[-1])
    pgn_file = io.StringIO(df_row["pgn"])       # PGN as a file
    game = chess.pgn.read_game(pgn_file) # Read PGN and put into game
    board = game.board()            # "board" of a game
    
    precheck = False                # Is en passant possible
    moved = False
    move_colour = ""
    
    # Find only pawn moves in game
    for move in game.mainline_moves():
        san = board.san(move)
        turn = board.turn
        move_piece = san[0]
        
        if precheck:
            move_ep_piece = san[:2]
            # print(move_ep_piece)
            # if (re.search("[abcdefgh]x", move_ep_piece) != None):
            # if move_ep_piece in "axbxcxdxexfxgxhx":
            if move_ep_piece in ep_set:
            # if ep_lookup.get(move_ep_piece, None):
            # if [e in "axbxcxdxexfxgx" for e in [move_ep_piece] if e in "axbxcxdxexfxgxhx"]:
            # if move_ep_piece.startswith(("ax", "bx", "cx", "dx", "ex", "fx", "gx", "hx")):
                moved = board.is_en_passant(move)
                # If ep actually happened, immediately return True
                break
            else:
                break
        
        # if [e in "abcdefgh" for e in [move_piece] if e in "abcdefgh"]:
        # if move_piece.startswith(("a", "b", "c", "d", "e", "f", "g", "h")):
        # if (re.search("^[abcdefgh]", move_piece) != None):
        # if move_piece in "abcdefgh":
        # if check_lookup.get(move_piece, None):
        if move_piece in check_set:
            # Push the move before checking the board
            board.push(move)
            precheck = board.has_legal_en_passant()

            # Return True the moment a potential ep move has been found
            if precheck:
                if turn:
                    move_colour = 'Black'
                else:
                    move_colour = 'White'
                # break
        else:
            board.push(move)
    
    df_row["ep_opportunity"] = precheck
    df_row["ep_happened"] = moved
    df_row["ep_colour"] = move_colour
    return df_row

# Save PGN column from dataframe
# full_pgn = chess_filter['pgn']


def test_time():
    start = time.time()
    # Add Boolean value to dataframe for whether an ep opportunity arose
    chess_filter = chess_filter2.apply(check_pgn, axis=1)
    end = time.time()
    print(end - start)

    chess_filter = chess_filter[chess_filter['ep_opportunity'] == True]
    chess_filter = chess_filter.drop(['ep_opportunity'], axis=1)

loop_start = time.time()
for i in range(10):
    test_time()
loop_end = time.time()
print((loop_end - loop_start)/10)


# start = time.time()
# # Add Boolean value to dataframe for whether an ep opportunity arose
# chess_filter = chess_filter2.apply(check_pgn, axis=1)
# end = time.time()
# print(end - start)

# chess_filter = chess_filter[chess_filter['ep_opportunity'] == True]
# chess_filter = chess_filter.drop(['ep_opportunity'], axis=1)


# # Filter out games where ep didn't happen
# chess_filter = chess_filter[chess_filter['ep_opportunity'] == True]
# chess_filter = chess_filter.drop(['ep_opportunity'], axis=1)

# # Re-save PGN column from updated dataframe
# full_pgn = chess_filter['pgn']

# print("second function")
# start = time.time()
# # Add Boolean value to dataframe for whether an ep capture actually happened
# chess_filter['ep_happened'] = full_pgn.apply(check_pgn_happened)
# end = time.time()
# print(end - start)

# print("third function")
# start = time.time()
# # Add column indicating the colour of the player who had the ep choice
# chess_filter['ep_colour'] = full_pgn.apply(check_pgn_turn)
# end = time.time()
# print(end - start)

319.42498207092285


In [162]:
# test 1k
# integrated 
# 4.990805387496948
# 4.859334945678711
# 5.083925724029541
# 4.817258596420288
# 4.952197551727295

# separate function
# 6.857896327972412
# 6.422466993331909
# 6.539485454559326
# 6.5865678787231445
# 6.551251411437988

# averages (ep take regex)
# regex: 4.9912710189819336
# s in word: 4.795914840698242
# list comp: 4.807268524169922
# startswith: 4.7986653089523315

# ep take average
# regex: 4.9912710189819336
# s in word: 4.8416884899139404
# s in array: 4.9870720863342285
# list comp: 4.915169954299927
# startswith: 4.824119234085083

# s in set: 4.8150769710540775
# s in set(str): 4.7867265224456785
# reverse lookup: 4.839635062217712


# 1k test 2 (no e.p. on s in word)
# lookup table 4.850662207603454
# s in set 4.884909176826477

# no ep
# list comp 4.871395707130432
# lookup table 4.861196708679199
# s in set 4.805133032798767
# s in set(str) 4.817056965827942

# TEST 5k
# set - reverse: 23.782688903808594
# set - set: 23.74259955883026

# all in one
# 319.42498207092285
# 442.72376894950867
chess_filter3 = chess_filter[chess_filter['ep_happened'] == True]
chess_filter3 = chess_filter3.drop(['ep_happened'], axis=1)
print(len(chess_filter))
print(len(chess_filter3))
# print(chess_filter.head(50))

5366
1673


In [ ]:
# legacy check

# first function
# 324.53476786613464
# second function
# 29.074277639389038
# 5366

# with regex:

# first function
# 25.838924646377563
# second function
# 32.171029806137085
# third function
# 24.009830713272095

# 318s

# case switch
# 324.53476786613464

# regex
# 25.838924646377563

# startswith tuples
# 22.949651956558228

# startswith (no fen)
# 22.792099952697754

# string "in"
# 22.84489679336548

# list comprehension
# 22.962693452835083

#### Cleaning data for EP - Advantage for ep

Analysing game - finding if ep move would have given an edvantage


Split into cases: ep happened and ep didn't happen


NOT DONE YET

In [ ]:
# def find_fen_ep_opportunity(pgn_in):
#     '''
#     Returns FEN of board when the ep opportunity arises
#     '''
#     pgn = io.StringIO(pgn_in)       # PGN as a file
#     game = chess.pgn.read_game(pgn) # Read PGN and put into game
#     board = game.board()            # "board" of a game
    
#     precheck = False                # Is en passant possible
#     moved = False                   # Did ep happen

#     fen_before_ep = ""
#     fen_after_ep = ""
#     reached_ep_opp = 0

#     # Find only pawn moves in game
#     for move in game.mainline_moves():
#         # B bishop Q queen K king N knight R rook O castle ELSE pawn
#         turn = board.turn
#         san = board.san(move)
#         move_piece = san[0]

#         match move_piece:
#             # Ignore if not pawn
#             case "K" | "Q" | "B" | "N" | "R" | "O":
#                 board.push(move)
#                 continue

#             # Pawn
#             case _:
#                 # Find the ep opportunity move
#                 # Push the move before checking the board
#                 moved = board.is_en_passant(move)
#                 board.push(move)
#                 precheck = board.has_legal_en_passant()

                
#                 if moved:
#                     fen_after_ep = board.fen()
#                     # Break out of loop
#                     return fen_before_ep, fen_after_ep
#                 elif reached_ep_opp > 0:
#                     # If ep wasn't taken, find fen for if it was actually taken

#                     # LILA STILL GOT TO DO THIS
#                     # (Find what the ep move would be and add it to the pgn if ep hadn't actually been chosen)
                    
#                 elif precheck:
#                     fen_before_ep = board.fen()
#                     reached_ep_opp += 1
                
#     # If pgn without ep opportunity was passed in, return None
#     return fen_before_ep, fen_after_ep





# def find_ep_advantage(pgn_in, ep_happened):
#     '''
#     Returns advantage given from ep move
#     '''
#     pgn = io.StringIO(pgn_in)       # PGN as a file
#     game = chess.pgn.read_game(pgn) # Read PGN and put into game
#     board = game.board()            # "board" of a game

#     fen_before_ep = find_fen_ep_opportunity(pgn)


### Number of games with at least 1 possible en-passant move - ???:
## 4750
(maybe 4913 or even 4945)

### Number of games with an actual en-passant move - ???:
## 1566

#### Applying PCA


In [ ]:
# Replace Boolean and string variables with numbers
chess_data['ep_happened'] = chess_data['ep_happened'].replace({True:1, False:0})
chess_data['ep_colour'] = chess_data['ep_colour'].replace({'White':1, 'Black':0})
chess_data['rated'] = chess_data['rated'].replace({True:1, False:0})
chess_data['time_class'] = chess_data['time_class'].replace({'daily':3, 'rapid':2, 'blitz':1, 'bullet':0})

# Drop irrelevant columns, and save differently - as a dataframe including the rating and one without
chess_data_with_elo = chess_data.drop(['fen', 'pgn', 'move_list'], axis=1)
chess_data_without_elo = chess_data_with_elo.drop(['white_rating', 'black_rating'], axis=1)

print(chess_data_with_elo.head())
print('\n')
print(chess_data_without_elo.head())

    white_rating  black_rating  time_class  rated  ep_happened  ep_colour
40          1569          1546           3      1            0          1
48          1505          1635           3      1            0          1
55          1468          1870           3      1            0          1
56          1530          1459           3      1            0          0
81          1498          1540           2      1            0          1


    time_class  rated  ep_happened  ep_colour
40           3      1            0          1
48           3      1            0          1
55           3      1            0          1
56           3      1            0          0
81           2      1            0          1


##### Data Standardisation (For PCA)

Cite week 8 lab sheet in report? Heavily used their code to help

In [ ]:
# THIS PART IS COPIED AND PASTED FROM WEEK 8 PCA SOLUTIONS:
def sort_eigenvalues(eigenvalues, eigenvectors):
    idx = eigenvalues.argsort()[::-1]   
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:,idx]
    return eigenvalues, eigenvectors

# THE REST IS HEAVILY INFLUENCED BY WEEK 8 PCA SOLUTIONS

# Standardise chess_data_without_elo so that it follows the Standard normal distribution
for col in chess_data_without_elo.columns:
        chess_data_without_elo[col] = (chess_data_without_elo[col] - chess_data_without_elo[col].mean()) / chess_data_without_elo[col].std()

# # Create matrix storing covariances of the chess dataframe features
# covariances = np.cov(chess_data_without_elo.values.T)

# # Record eigenvalues and eigenvectors
# eigenvalues, eigenvectors = np.linalg.eig(covariances)
# eigenvalues, eigenvalues = sort_eigenvalues(eigenvalues, eigenvectors)

# result = chess_data_without_elo.dot(eigenvectors[:,:2])

# sns.scatterplot(x= result[0], y = result[1], hue=diagnosis)
# plt.xlabel('PC1')
# plt.ylabel('PC2')




# Lila Can we instead use this? :
pca = PCA(n_components=3).fit(chess_data_without_elo.values)
pca_result = pca.transform(chess_data_without_elo.values)
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1])
plt.xlabel('PC1')
plt.ylabel('PC2')

ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values